<a href="https://colab.research.google.com/github/Maria-Evlakhova/ClinicalRecommendations/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BB%D0%B8%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Инструменты:


*   gigachain - основной инструмент построения пайплайнов с большими языковыми моделями;
*   faiss-cpu - используем векторное хранилище FAISS для размещения чанков (фрагментов текстов для контекста)и эмбеддингов (векторных представлений) этих чанков;
*   sentence-transformers - библиотека для локального использования эмбеддинговых моделей с ресурса Hugging Face;
*   rank_bm25 - библиотека для работы с алгоритмами векторизации BM25 (вариации TF-IDF);
*   gigachain-community unstructured[all-docs] - библиотека для распознавания всех типов документов.




In [1]:
!pip install gigachain gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Создание фрагментов на основе PDF
Для подготовки фрагментов используется RecursiveCharacterTextSplitter. Также задается приблизительный размер фрагмента (chunk_size) и количество символов перекрытия (chunk_overlap). Сплиттер можно передать в метод load_and_split(). На выходе получаем список документов Document.

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=500)
pdf_file = 'КР359_3.pdf'
loader = UnstructuredPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

Смотрим количество фрагментов

In [2]:
len(splitted_data)

822

Выведем на экран два соседних фрагмента, чтобы показать перекрытие.

In [3]:
splitted_data[10]

Document(metadata={'source': 'КР359_3.pdf'}, page_content='основанных на использовании природных лечебных факторов.............................................. 85\n\n5. Профилактика\n\nи\n\nдиспансерное\n\nнаблюдение, медицинские\n\nпоказания\n\nпротивопоказания к применению методов профилактики........................................................86\n\n6. Организация оказания медицинской помощи..........................................................................90\n\n7. Дополнительная информация (в том числе факторы, влияющие на исход заболевания\n\nили состояния)................................................................................................................................... 93\n\n8. Критерии оценки качества медицинской помощи..................................................................96\n\nСписок литературы...........................................................................................................................99\n\nПриложение А1. Сост

In [4]:
splitted_data[11]

Document(metadata={'source': 'КР359_3.pdf'}, page_content='или состояния)................................................................................................................................... 93\n\n8. Критерии оценки качества медицинской помощи..................................................................96\n\nСписок литературы...........................................................................................................................99\n\nПриложение А1. Состав рабочей группы по разработке и пересмотру клинических\n\nрекомендаций...................................................................................................................................146\n\nПриложение А2. Методология разработки клинических рекомендаций..............................151\n\nПриложение АЗ. Справочные материалы, включая соответствие показаний к применению\n\nи противопоказаний, способов применения и доз лекарственных препаратов, инструкции')

## Создание векторных представлений и векторного хранилища
В этом туториале опять используем модель paraphrase-multilingual-mpnet-base-v2 для векторизации фрагментов.

В качестве хранилища используется FAISS.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document

Создаем ретривер и векторное хранилище для фрагментов текста.

In [6]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

<timed exec>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 5min 33s, sys: 21.8 s, total: 5min 54s
Wall time: 6min 8s


Задаем параметры извлечения, на запрос должны возвращаться 5 фрагментов, наиболее близких по смыслу.

In [7]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

## Создание генеративной части пайплайна и составление цепочки RAG системы
Необходимо использовать авторизационные данные для подключения к GigaChat API.

In [8]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

Импортируем необходимые компоненты

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

Создадим объект GigaChat и подготовим промпт для вопросно-ответной системы.

In [10]:

llm = GigaChat(credentials=auth,
              model='GigaChat:latest',
               verify_ssl_certs=False,
               profanity_check=False)
prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

Создадим цепочку create_stuff_documents_chain, которая будет частью вопросно-ответной цепочки. Это нужно для того, чтобы подавать фрагменты текстов из векторной БД в промпт языковой модели. Промпт представляет из себя форматированную строку, а фрагменты являются экземплярами класса Document. Чтобы не писать код по извлечению атрибута page_content из Document, используем цепочку create_stuff_documents_chain, где это автоматизировано.

In [11]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
    )

Создадим вопросно-ответную цепочку с помощью функции create_retrieval_chain().

In [12]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

## Запустим вопросно-ответную систему

In [13]:
q1 = 'Как лечить астму?'

In [14]:

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

In [15]:

resp1


{'input': 'Как лечить астму?',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='6.\n\nастмы) Выполнено проведение окисгенотерапии(при сатурации менее 90%) (при обострении бронхиальной астмы при отсутствии .медицинских противопоказаний)\n\n7. Выполнено назначение лекарственных препаратов групп: адренергические средства для ингаляционного введения или адренергические средства в комбинации с кортикостероидами или другими препаратами, исключая антихолинергические средства (при отсутствии медицинских противопоказаний)\n\nДа/Нет\n\n8. Выполнено назначение лекарственных препаратов действия) групп: или (системного глюкокортикоидыов кортикостероидов (ингаляционно), кроме случаев применения адренергических с в другими глюкокортикоидамикортикостероидами препаратами, исключая кроме средства) и/или блокаторов лейкотриеновых рецепторов антагонисты (в зависимости от медицинских при медицинских противопоказаний)\n\nсредств\n\nкомбинации или\n\nантихолинергическихе\n\nпоказаний\n

In [16]:
q2 = 'Какие диагностические исследования необходимы?'

In [17]:
resp2 = retrieval_chain.invoke(
    {'input': q2}
)

In [18]:
resp2

{'input': 'Какие диагностические исследования необходимы?',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='всех симптомов не обязательно. Важнейшим фактором диагностики является\n\nтщательный\n\nсбор\n\nанамнеза,\n\nкоторый\n\nукажет\n\nна\n\nпричины\n\nвозникновения,\n\nпродолжительность и разрешение симптомов, наличие аллергических заболеваний\n\n(атопический дерматит, аллергически ринит, пищевая аллергия) у пациента и его\n\nкровных родственников, причинно-следственные особенности возникновения признаков\n\nболезни и ее обострений с учетом основных клинических признаков, повышающих\n\nвероятность наличия БА у пациента.\n\nПри сборе анамнеза следует обратить внимание на наличие типичного\n\nсимптомокомплекса:\n\nповторяющиеся\n\nприступы\n\nкашля,\n\nсвистящее\n\nдыхание,\n\nзатрудненное дыхание или чувство стеснения в груди, вызванные различными триггерами\n\n(такими как респираторная инфекция, табачный дым, контакт с животными или\n\nпыльцой и т.д.; физичес

In [19]:
q3 = 'Диагностика астмы'

In [20]:
resp3 = retrieval_chain.invoke(
    {'input': q3}
)

In [21]:
resp3

{'input': 'Диагностика астмы',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='дыхания в анамнезе диагноз бронхиальной астмы становится более вероятным если:\n\nсвистящее дыхание или кашель появляются на фоне физической нагрузки, смехе,\n\nплаче или в отсутствие явных признаков респираторной инфекции\n\nналичие сопутствующих аллергических заболеваний (атопический дерматит,\n\nаллергический ринит) у самого ребенка или наличие сенсибилизации к аллергенам или\n\nастмы у родственников первой линии;\n\nклиническое улучшение в течение 2-3 месячного курса низкодозной терапии\n\nингаляционными глюкокортикостероидами (группа АТХ КОЗВА: Кортикостероиды) в\n\nсочетании с коротко действующими бета2-агонистами (некоторые препараты из\n\nгруппы АТХ КОЗ АС: Селективные\n\nбета2-адреномиметики,\n\nхарактеризующиеся\n\nпродолжительностью действия менее 6 часов) по требованию. При отмене терапии\n\nнаступает ухудшение.\n\nисключены альтернативные заболевания [21].\n\n162\n\nКлини

In [22]:
q4 = 'Факторы, влияющие на развитие и проявление астмы'

In [23]:
resp4 = retrieval_chain.invoke(
    {'input': q4}
)

In [24]:
resp4

{'input': 'Факторы, влияющие на развитие и проявление астмы',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='дыхания в анамнезе диагноз бронхиальной астмы становится более вероятным если:\n\nсвистящее дыхание или кашель появляются на фоне физической нагрузки, смехе,\n\nплаче или в отсутствие явных признаков респираторной инфекции\n\nналичие сопутствующих аллергических заболеваний (атопический дерматит,\n\nаллергический ринит) у самого ребенка или наличие сенсибилизации к аллергенам или\n\nастмы у родственников первой линии;\n\nклиническое улучшение в течение 2-3 месячного курса низкодозной терапии\n\nингаляционными глюкокортикостероидами (группа АТХ КОЗВА: Кортикостероиды) в\n\nсочетании с коротко действующими бета2-агонистами (некоторые препараты из\n\nгруппы АТХ КОЗ АС: Селективные\n\nбета2-адреномиметики,\n\nхарактеризующиеся\n\nпродолжительностью действия менее 6 часов) по требованию. При отмене терапии\n\nнаступает ухудшение.\n\nисключены альтернативные за

## Загрузка кодов МКБ из xlsx-файла

In [25]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader('mkb10.xlsx')
splitted_xlsx = loader.load_and_split(text_splitter)

Смотрим количество фрагментов

In [26]:
len(splitted_xlsx)

310

Увидим содержимое отдельного фрагмента

In [27]:
splitted_xlsx[5]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='Некоммерческая организация "Ассоциация онкологических организаций Сибири и Дальнего Востока Да 2025-01-24 12:51:34.887000 Применяется 355_5 Рак поджелудочной железы C25, D37.7 Взрослые Ассоциация онкологов России, Общероссийская общественная организация "Российское общество хирургов", Общероссийская общественная организация «Российское общество клинической онкологии», Общероссийская общественная организация содействия развитию лучевой диагностики и терапии "Российское общество рентгенологов и радиологов" Да 2025-01-23 16:31:25.343000 Применяется 460_4 Рак тела матки и саркомы матки C54, C55, D07.0 Взрослые Ассоциация онкологов России, Общероссийская общественная организация «Российское общество клинической онкологии», Общероссийская общественная организация «Российское общество специалистов по профилактике и лечению опухолей репродуктивной системы» Да 2025-01-23 14:42:38.013000 Применяется 144_2 Множественная миелома C90.0 Взрос

In [39]:
splitted_xlsx1 = loader.load_and_split()

In [40]:
len(splitted_xlsx1)

41

In [41]:
splitted_xlsx1[3]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='– реаниматологов", Межрегиональная ассоциация по клинической микробиологии и антимикробной химиотерапии, Межрегиональная общественная организация «Альянс клинических химиотерапевтов и микробиологов», Межрегиональная общественная организация врачей и медицинских сестер «Сепсис Форум», Общероссийская общественная организация «Российское общество скорой медицинской помощи», Национальная ассоциация специалистов по тромбозам, клинической гемостазиологии и гемореологии, Российская Ассоциация специалистов по лечению хирургических инфекций Да 2024-12-27 15:31:08.733000 Применяется 590_2 Психические и поведенческие расстройства, вызванные употреблением психоактивных веществ Абстинентное состояние (синдром отмены) с делирием F10.4, F11.4, F13.4, F19.4 Взрослые Общественная организация "Российское общество психиатров", Общероссийская общественная организация "Федерация анестезиологов и реаниматологов" Да 2024-12-27 14:56:58.420000 Применяе

## Создание векторных представлений и векторного хранилища
Используется загрузчик с настройками по умолчанию. Создаем очередное векторное хранилище.

In [47]:
vector_store_xlsx = FAISS.from_documents(splitted_xlsx1, embedding=embedding)

## Получаем ответ на вопрос по кодам МКБ

In [48]:
q5 = 'Какой код МКБ-10 у дистонии?'

In [49]:
resp5 = retrieval_chain.invoke(
    {'input': q5}
)


In [50]:
resp5

{'input': 'Какой код МКБ-10 у дистонии?',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='Степень тяжести\n\nЛегкое/умеренное\n\nТяжелое 1\n\nобострения\n\nСимптомы\n\nИзмененное сознание\n\nСатурация (8р02)2 Речь3 Пульс\n\nчдд Центральный цианоз Интенсивность хрипов\n\nНет\n\n>95% Предложениями <100 ударов в минуту\n\n<40 в минуту Отсутствует Вариабельна\n\nВозбуждение, спутанность сознания или сонливость <92% Словами >180 ударов в минуту (0-3 года) >150 ударов в минуту (4-5 лет) >40 в минуту Чаще всего присутствует Возможно наличие «немой»\n\n19\n\n______________________________ ________________________ грудной клетки____________ Примечание.1 Каждый из этих симптомов указывает на наличие тяжелого обострения.\n\n2Измерение\n\nсатурации\n\nпроводится\n\nдо\n\nприема\n\nКДБА\n\nи/или\n\nкислородотерапии.\n\n3 Необходимо учитывать возраст и возможности ребенка\n\nПод\n\nастматическим\n\nстатусом\n\nпонимают\n\nэпизод\n\nострой\n\nдыхательной\n\nнедостаточности (ОД